# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,22.47,84,96,4.27,US,1724024882
1,1,huayacocotla,20.5500,-98.4833,16.26,85,80,2.39,MX,1724025171
2,2,namibe,-15.1961,12.1522,16.87,86,10,2.15,AO,1724025172
3,3,yellowknife,62.4560,-114.3525,21.34,60,75,4.12,CA,1724025173
4,4,zaozersk,69.4005,32.4476,13.96,66,59,2.53,RU,1724025174


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
my_weather_conditions = city_data_df.loc[(city_data_df['Max Temp'] > 22.5) & (city_data_df['Max Temp'] < 23), :]
my_weather_conditions

# Drop any rows with null values
my_weather_conditions = my_weather_conditions.dropna(how = 'any') 
                       
# Display sample data
my_weather_conditions.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,afaahiti,-17.7500,-149.2833,22.92,80,84,11.99,PF,1724025246
69,69,tamanrasset,22.7850,5.5228,22.95,68,20,3.60,DZ,1724025254
103,103,santa elena,-2.2333,-80.8500,22.75,83,40,5.14,EC,1724025177
119,119,ribeira grande,38.5167,-28.7000,22.65,86,4,1.79,PT,1724025316
155,155,tonekabon,36.8163,50.8738,22.51,89,88,1.07,IR,1724025358


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_weather_conditions.copy()
hotel_df.reset_index(inplace=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel name'] = ""

# Display sample data
hotel_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name
0,62,62,afaahiti,-17.7500,-149.2833,22.92,80,84,11.99,PF,1724025246,
1,69,69,tamanrasset,22.7850,5.5228,22.95,68,20,3.60,DZ,1724025254,
2,103,103,santa elena,-2.2333,-80.8500,22.75,83,40,5.14,EC,1724025177,
3,119,119,ribeira grande,38.5167,-28.7000,22.65,86,4,1.79,PT,1724025316,
4,155,155,tonekabon,36.8163,50.8738,22.51,89,88,1.07,IR,1724025358,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row['Lng']
    latitude = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
afaahiti - nearest hotel: Omati Lodge
tamanrasset - nearest hotel: فندق الأمان
santa elena - nearest hotel: Hotel El Gran Jeshua
ribeira grande - nearest hotel: Hospedaria JSF
tonekabon - nearest hotel: ساختمان پارسیان ۲
rockland - nearest hotel: Cullens Hotel
fort st. john - nearest hotel: Fort St. John Motor Inn
nova vicosa - nearest hotel: Pousada Pontal da Barra
skhirate - nearest hotel: Amphitrite Palace
chilpancingo - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
kavala - nearest hotel: Hotel Esperia
booue - nearest hotel: La splendeur de l'équateur
yeraliyev - nearest hotel: Kuryk
puerto rico - nearest hotel: Suizo


,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel name,Hotel Name
0,62,62,afaahiti,-17.7500,-149.2833,22.92,80,84,11.99,PF,1724025246,,Omati Lodge
1,69,69,tamanrasset,22.7850,5.5228,22.95,68,20,3.60,DZ,1724025254,,فندق الأمان
2,103,103,santa elena,-2.2333,-80.8500,22.75,83,40,5.14,EC,1724025177,,Hotel El Gran Jeshua
3,119,119,ribeira grande,38.5167,-28.7000,22.65,86,4,1.79,PT,1724025316,,Hospedaria JSF
4,155,155,tonekabon,36.8163,50.8738,22.51,89,88,1.07,IR,1724025358,,ساختمان پارسیان ۲


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)